### DataFrame 생성 방법
1. Sparksession을 통하여
    - 외부 파일을 읽어 들일 때 SparkSession을 통하여 DataFrame으로 읽어들임
    - Spark SQL 쿼리를 실행하여 다른 DataFrame으로 변환
2. Rdd로부터 변환
    - SparkContext에서 RDD를 생성
    - RDD를 DataFrame으로 변환

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
import pyspark.sql

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.master("local").getOrCreate()

In [11]:
menuDF = spark  \
         .read   \
         .format("csv")  \
         .option("header","true")  \
         .option("inferSchema", "true")  \
         .load("C://spark/data/menu.csv")

In [12]:
menuDF.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Serving Size: string (nullable = true)
 |-- Calories: integer (nullable = true)
 |-- Calories from Fat: integer (nullable = true)
 |-- Total Fat: double (nullable = true)
 |-- Total Fat (% Daily Value): integer (nullable = true)
 |-- Saturated Fat: double (nullable = true)
 |-- Saturated Fat (% Daily Value): integer (nullable = true)
 |-- Trans Fat: double (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- Cholesterol (% Daily Value): integer (nullable = true)
 |-- Sodium: integer (nullable = true)
 |-- Sodium (% Daily Value): integer (nullable = true)
 |-- Carbohydrates: integer (nullable = true)
 |-- Carbohydrates (% Daily Value): integer (nullable = true)
 |-- Dietary Fiber: integer (nullable = true)
 |-- Dietary Fiber (% Daily Value): integer (nullable = true)
 |-- Sugars: integer (nullable = true)
 |-- Protein: integer (nullable = true)
 |-- Vitamin A (% Daily Value): integer (nul

In [13]:
menuDF.show()

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+------

- RDD를 DataFrame으로 변환
    - sparkSession의 createDataFrame 사용

### DataFrame 스키마 관련 기본 함수들
- printSchema() : 스키마를 출력
- show(n) : 처음 (n)행을 출력
- withColumnRenamed(현컬럼명, 새컬럼명) : 현 컬럼의 이름을 새 컬럼 이름으로 변경

In [14]:
sc = spark.sparkContext

In [16]:
linesDF = spark  \
          .read  \
          .format("csv")  \
          .option("encoding", "EUC-kr")  \
          .option("sep", "\n")  \
          .load("C://spark/data/review.txt")

In [18]:
linesDF.take(2)

[Row(_c0='배우의 국적을 의심하게 만듬.'),
 Row(_c0='여주를 잘 알려지지 않은 배우로 캐스팅한게 정말 탁월했습니다! 진짜 일본인이 연기하는 줄 착각할 정도로 연기가 정말 좋았네요! 최희서 배우님 진짜 앞으로 더욱 기대되는 배우입니다!')]

In [19]:
import re

In [20]:
linesRDD = linesDF.rdd

In [21]:
linesRDD.take(3)

[Row(_c0='배우의 국적을 의심하게 만듬.'),
 Row(_c0='여주를 잘 알려지지 않은 배우로 캐스팅한게 정말 탁월했습니다! 진짜 일본인이 연기하는 줄 착각할 정도로 연기가 정말 좋았네요! 최희서 배우님 진짜 앞으로 더욱 기대되는 배우입니다!'),
 Row(_c0='역사를 잊은 민족에게 미래는 없다')]

In [22]:
wcRDD = linesDF  \
.rdd  \
.map(lambda line: line[0])  \
.flatMap(lambda line: re.split("\W+", line))  \
.filter(lambda word: len(word) > 0)  \
.map(lambda word: (word, 1))  \
.reduceByKey(lambda v1, v2: v1 + v2)  \
.sortBy(lambda wc: wc[1], False)

In [23]:
wcRDD.take(50)

[('영화', 1346),
 ('너무', 898),
 ('정말', 698),
 ('진짜', 536),
 ('잘', 518),
 ('연기', 493),
 ('더', 342),
 ('박열', 332),
 ('연기가', 329),
 ('이제훈', 312),
 ('꼭', 285),
 ('수', 277),
 ('배우들', 273),
 ('배우들의', 271),
 ('역사를', 260),
 ('영화를', 259),
 ('이', 255),
 ('연기도', 250),
 ('이준익', 219),
 ('그', 213),
 ('그리고', 212),
 ('좀', 209),
 ('좋은', 204),
 ('영화는', 202),
 ('한', 197),
 ('후미코', 196),
 ('좋았습니다', 192),
 ('영화가', 190),
 ('봤습니다', 188),
 ('보고', 184),
 ('다', 179),
 ('그냥', 175),
 ('재밌게', 174),
 ('최희서', 166),
 ('연기는', 165),
 ('감사합니다', 165),
 ('있는', 164),
 ('이런', 160),
 ('본', 154),
 ('영화입니다', 153),
 ('많이', 152),
 ('것', 150),
 ('하는', 145),
 ('대해', 142),
 ('보는', 142),
 ('봤는데', 140),
 ('박열이라는', 131),
 ('역시', 130),
 ('또', 129),
 ('다시', 125)]

In [24]:
wc50RDD = sc.parallelize(wcRDD.take(50))

In [25]:
wcDF = spark.createDataFrame(wc50RDD)

In [26]:
wcDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [28]:
wcDF.show(3)

+----+----+
|  _1|  _2|
+----+----+
|영화|1346|
|너무| 898|
|정말| 698|
+----+----+
only showing top 3 rows



In [29]:
wcDFnamed = wcDF.withColumnRenamed("_1", "words")  \
            .withColumnRenamed("_2", "freq")

In [30]:
wcDFnamed.printSchema()

root
 |-- words: string (nullable = true)
 |-- freq: long (nullable = true)



In [33]:
wcDFnamed.show(3)

+-----+----+
|words|freq|
+-----+----+
| 영화|1346|
| 너무| 898|
| 정말| 698|
+-----+----+
only showing top 3 rows



- withColumn(컬럼명,계산식) : 계산식에 의하여 만들어지는 새로운 컬럼을 생성

In [36]:
import pyspark.sql.functions as F

In [37]:
wcDFnamed.select(F.col("words")).show(5)

+-----+
|words|
+-----+
| 영화|
| 너무|
| 정말|
| 진짜|
|   잘|
+-----+
only showing top 5 rows



In [40]:
wcDFlength = wcDFnamed.withColumn("wordLen",F.length("words"))

In [41]:
wcDFlength.show(3)

+-----+----+-------+
|words|freq|wordLen|
+-----+----+-------+
| 영화|1346|      2|
| 너무| 898|      2|
| 정말| 698|      2|
+-----+----+-------+
only showing top 3 rows



### DataFrame Method

- DataFrame의 대표적인 함수들
    - select
    - filter
    - where
    - orderBy
    - GroupBy
    - agg
- 이들 methods를 수행하기 위해서는
    - spark.sql.functions을 import 해서
    - spark.sql.functions가 제공하는 함수들을 사용
        - col
        - length

#### .select(컬럼명):
- 컬럼들만을 선택
- 새로운 DataFrame을 생성

In [42]:
wcDFlength.show(3)

+-----+----+-------+
|words|freq|wordLen|
+-----+----+-------+
| 영화|1346|      2|
| 너무| 898|      2|
| 정말| 698|      2|
+-----+----+-------+
only showing top 3 rows



In [44]:
type(wcDFlength)

pyspark.sql.dataframe.DataFrame

In [45]:
wcDFlength.select(F.col("words")).show(2)

+-----+
|words|
+-----+
| 영화|
| 너무|
+-----+
only showing top 2 rows



In [46]:
wcDFlength.select(wcDFlength.words).show(2)

+-----+
|words|
+-----+
| 영화|
| 너무|
+-----+
only showing top 2 rows



In [47]:
wcDFlength.select("words").show(2)

+-----+
|words|
+-----+
| 영화|
| 너무|
+-----+
only showing top 2 rows



In [50]:
wcDFlength.select(F.col("wordLen"), F.col("freq")).show()

+-------+----+
|wordLen|freq|
+-------+----+
|      2|1346|
|      2| 898|
|      2| 698|
|      2| 536|
|      1| 518|
|      2| 493|
|      1| 342|
|      2| 332|
|      3| 329|
|      3| 312|
|      1| 285|
|      1| 277|
|      3| 273|
|      4| 271|
|      3| 260|
|      3| 259|
|      1| 255|
|      3| 250|
|      3| 219|
|      1| 213|
+-------+----+
only showing top 20 rows



In [48]:
wcDFlength.filter(F.col("wordLen") <= 1).show()

+-----+----+-------+
|words|freq|wordLen|
+-----+----+-------+
|   잘| 518|      1|
|   더| 342|      1|
|   꼭| 285|      1|
|   수| 277|      1|
|   이| 255|      1|
|   그| 213|      1|
|   좀| 209|      1|
|   한| 197|      1|
|   다| 179|      1|
|   본| 154|      1|
|   것| 150|      1|
|   또| 129|      1|
+-----+----+-------+



In [52]:
wcDFlength.filter(wcDFlength.wordLen <= 1).show()

+-----+----+-------+
|words|freq|wordLen|
+-----+----+-------+
|   잘| 518|      1|
|   더| 342|      1|
|   꼭| 285|      1|
|   수| 277|      1|
|   이| 255|      1|
|   그| 213|      1|
|   좀| 209|      1|
|   한| 197|      1|
|   다| 179|      1|
|   본| 154|      1|
|   것| 150|      1|
|   또| 129|      1|
+-----+----+-------+



#### .filter(조건식)
- 조건식을 만족시키는 레코드들만을 선택
- (새로운 DataFrame을 생성)

#### .where(조건식)
- filter와 동일 기능
- " "안에 조건식을 추가

In [54]:
wcDFlength.where("wordLen <= 1").show()

+-----+----+-------+
|words|freq|wordLen|
+-----+----+-------+
|   잘| 518|      1|
|   더| 342|      1|
|   꼭| 285|      1|
|   수| 277|      1|
|   이| 255|      1|
|   그| 213|      1|
|   좀| 209|      1|
|   한| 197|      1|
|   다| 179|      1|
|   본| 154|      1|
|   것| 150|      1|
|   또| 129|      1|
+-----+----+-------+



#### .groupby(컬럼명) : 해당 컬럼의 값이 같은 레코드들을 모두 모음
#### .agg(계산식) : 계산식의 값을 합계 처리
#### .orderBy(컬럼명) : 컬럼명의 값에 따라 레코드들을 정렬

In [59]:
wordLengthFrequency = wcDFlength  \
.groupBy(F.col("wordLen"))  \
.agg(F.count(F.col("freq")), F.avg(F.col("freq")), F.sum(F.col("freq"))) \
.orderBy(F.col("wordLen"))

In [60]:
wordLengthFrequency.show()

+-------+-----------+------------------+---------+
|wordLen|count(freq)|         avg(freq)|sum(freq)|
+-------+-----------+------------------+---------+
|      1|         12|242.33333333333334|     2908|
|      2|         17|354.47058823529414|     6026|
|      3|         15|223.13333333333333|     3347|
|      4|          2|             229.5|      459|
|      5|          4|            160.25|      641|
+-------+-----------+------------------+---------+

